In [1]:
 import numpy as np
 from scipy.integrate import solve_ivp
 from numba import njit
 from jitr import (
     ProjectileTargetSystem,
     InteractionMatrix,
     ChannelData,
     LagrangeRMatrixSolver,
     woods_saxon_potential,
     delta,
     smatrix,
     schrodinger_eqn_ivp_order1,
     FreeAsymptotics
 )


In [2]:
 @njit
 def interaction(r, *args):
     (V0, W0, R0, a0) = args
     return woods_saxon_potential(r, (V0, W0, R0, a0))


In [3]:
sys = ProjectileTargetSystem(
    np.array([939.0]),
    np.array([5 * (2 * np.pi)]),
    l=np.array([0]),
    Ztarget=40,
    Zproj=0,
    nchannels=1,
)

# Lagrange-Mesh solver, don't set the energy
solver_lm = LagrangeRMatrixSolver(5, 1, sys, ecom=None)

# use same interaction for all channels
interaction_matrix = InteractionMatrix(1)
interaction_matrix.set_local_interaction(interaction, 0, 0)

# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness

params = (V0, W0, R0, a0)

/home/kyle/mambaforge/envs/om/lib/python3.10/site-packages/numba/core/dispatcher.py:289: UserWarning: Numba extension module 'numba_scipy' failed to load due to 'ValueError(No function '__pyx_fuse_0pdtr' found in __pyx_capi__ of 'scipy.special.cython_special')'.
  entrypoints.init_all()


In [4]:
Ecom = 14.1
ch = np.array(sys.build_channels(Ecom))
solver_lm.set_energy(Ecom)

In [5]:
F = solver_lm.kernel.single_channel_free_matrix(ch[0])

In [6]:
R_lm, S_lm, x, uext_prime_boundary = solver_lm.solve(
    interaction_matrix, ch, args=params, ecom=Ecom
)

In [7]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    ch[0].domain,
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-9,
    rtol=1.0e-9,
).sol

In [8]:
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
print(R_rk)
S_rk = smatrix(R_rk, a, ch[0].l, ch[0].eta)
print(S_rk)


(-0.0021309110080514246+0.03561169340604122j)
(-0.05699944100839482-0.029794934193171304j)


In [9]:
R_lm

array([[-65831.47532017+17679.54889176j]])